<a href="https://colab.research.google.com/github/andyduong4321-debug/118S/blob/main/prompt_chaining_exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install once in Colab if needed:
# !pip install -q openai

import os
from typing import Optional

try:
    from openai import OpenAI
except Exception:
    OpenAI = None

MODEL = "gpt-5-mini"  # inexpensive model option
API_KEY = os.getenv("OPENAI_API_KEY")

def demo_response(prompt: str, tag: str = "") -> str:
    samples = {
        "classify": "Issue Type: Billing\nUrgency: Medium\nReason: Customer says they were charged twice for the same order.",
        "followup": "Thanks for reaching out. I can help with that. To verify the duplicate charge, please share: 1) your order number, 2) the date of the charge, and 3) the last 4 digits of the payment method.",
        "solution": "I found this looks like a duplicate charge case. Next step: verify the order number and charge date, then submit a billing review. If the duplicate is confirmed, issue a refund and send a 2-3 business day timeline.",
        "react_plan": "Plan:\n1. Read the task.\n2. Write a small Python function.\n3. Run it on a sample list.\n4. If it fails, fix it.\n5. Print the final result.",
        "react_code": "def count_vowels(text):\n    vowels = 'aeiouAEIOU'\n    return sum(1 for ch in text if ch in vowels)\n\nwords = ['apple', 'sky', 'education']\nfor w in words:\n    print(w, count_vowels(w))",
        "summary_before": "Solar energy is good because it comes from the sun and helps the environment. It is useful for homes and businesses.",
        "critique": "Critique:\n- Too general\n- Missing audience focus\n- No example\n- Could be more concise and structured\n\nImproved Summary:\nSolar energy gives homes and businesses a cleaner power option by using sunlight instead of fossil fuels. It can lower long-term electricity costs, but installation costs and weather limits should be considered.",
    }
    return samples.get(tag, "Demo mode output.")

def ask_model(prompt: str, tag: str = "", instructions: Optional[str] = None) -> str:
    if API_KEY and OpenAI is not None:
        client = OpenAI(api_key=API_KEY)
        kwargs = {"model": MODEL, "input": prompt}
        if instructions:
            kwargs["instructions"] = instructions
        resp = client.responses.create(**kwargs)
        return resp.output_text
    return demo_response(prompt, tag)

print("Using real API:", bool(API_KEY and OpenAI is not None))
print("Model:", MODEL)

Using real API: False
Model: gpt-5-mini


In [ ]:
customer_message = "Hi, I think I got charged twice for my last order. Can you help me fix it?"

step1_prompt = f'''
You are a support triage assistant.

Task:
Classify the customer's issue.

Customer message:
{customer_message}

Rules:
- Choose one issue type: Billing, Technical, Shipping, Account
- Choose one urgency: Low, Medium, High
- Give one short reason
- Keep the answer under 60 words
- Output format:
Issue Type: ...
Urgency: ...
Reason: ...
'''.strip()

step1_output = ask_model(step1_prompt, tag="classify")
print("STEP 1 OUTPUT\n")
print(step1_output)

STEP 1 OUTPUT

Issue Type: Billing
Urgency: Medium
Reason: Customer says they were charged twice for the same order.


In [ ]:
step2_prompt = f'''
You are a polite customer support agent.

Customer message:
{customer_message}

Classification from Step 1:
{step1_output}

Task:
Ask for the minimum missing information needed to continue.

Rules:
- Friendly tone
- Ask at most 3 questions
- Do not promise a refund yet
- Keep the answer under 80 words
'''.strip()

step2_output = ask_model(step2_prompt, tag="followup")
print("STEP 2 OUTPUT\n")
print(step2_output)

STEP 2 OUTPUT

Thanks for reaching out. I can help with that. To verify the duplicate charge, please share: 1) your order number, 2) the date of the charge, and 3) the last 4 digits of the payment method.


In [ ]:
step3_prompt = f'''
You are a support resolution assistant.

Customer message:
{customer_message}

Step 1 classification:
{step1_output}

Step 2 follow-up:
{step2_output}

Task:
Give the next support action.

Rules:
- Mention whether the case should be escalated
- Give a short action plan
- Keep the answer under 90 words
'''.strip()

step3_output = ask_model(step3_prompt, tag="solution")
print("STEP 3 OUTPUT\n")
print(step3_output)

STEP 3 OUTPUT

I found this looks like a duplicate charge case. Next step: verify the order number and charge date, then submit a billing review. If the duplicate is confirmed, issue a refund and send a 2-3 business day timeline.
